<center><img src="../images/DLI Header.png" alt="标题" style="width: 400px;"/></center>

# Hello Camera
### CSI（相机串行接口）相机

此 notebook 用于测试相机，以确保相机可在 Jetson Nano 上正常运行。相机应已插入 MIPI CSI 相机端口。确保相机镜头上未覆盖任何障碍物，如薄膜或封盖。

<center><img src="../images/csi_setup_sm.jpg" width=600/></center>

<div style="border:2px solid black; background-color:#e3ffb3; font-size:12px; padding:8px; margin-top: auto;"><i>
<h4><i>提示</i></h4>
如要在代码单元中执行 Python 或系统代码，请选择相应单元，然后单击窗口顶部的“Run”（运行）按钮。<br>您也可使用键盘快捷键运行代码单元：<strong>[SHIFT][ENTER]</strong>
</i></div>

### 检查设备是否可用
执行以下系统命令，以在 Jetson Nano 上列出所有视频设备。如未显示相机的设备 ID，请检查连接状态。所得输出应如下所示：
```text
crw-rw----+ 1 root video 81, 0 Jun  2 17:35 /dev/video0
```

In [ ]:
!ls -ltrh /dev/video*

### 创建相机对象

首先，执行以下 Python 代码单元从库中导入 `CSICamera` 类，以此创建相机对象。请注意，仅可创建一个 `CSICamera` 实例。

In [ ]:
from jetcam.csi_camera import CSICamera

camera = CSICamera(width=224, height=224)

然后，我们可以使用 `read` 方法从相机中捕获一帧图像。

In [ ]:
image = camera.read()

print(image.shape)

为 `camera` 调用 `read` 方法还会更新相机的内部 `value`。该值的 `shape` 会显示三个数值，分别表示像素高度、像素宽度和颜色通道数量。

In [ ]:
print(camera.value.shape)

### 创建小组件以查看图像流
我们可以创建“小组件”，以在 notebook 中显示此图像。如要查看图像，请将其从蓝绿红格式 (brg8) 转换为可供浏览器显示的格式 (jpeg)。

In [ ]:
import ipywidgets
from IPython.display import display
from jetcam.utils import bgr8_to_jpeg

image_widget = ipywidgets.Image(format='jpeg')

image_widget.value = bgr8_to_jpeg(image)

display(image_widget)

如果一切正常，您应会看到从相机捕获的图像。若显示图像，但图像模糊不清或颜色奇异，请检查并确保镜头上未覆盖任何保护膜或封盖。

现在，我们来看看相机中的实时流。设置相机的 `running` 值，以便在后台不断更新该值。这样，您便可附加“回调函数”以应对相机值出现变动的情形。

此处的“回调函数”是指 `update_image` 函数，可通过调用下方的 `observe` 方法进行附加。每当有可供处理的新图像时，系统就会执行 `update_image`，然后在小组件中将其显示出来。

In [ ]:
camera.running = True

def update_image(change):
    image = change['new']
    image_widget.value = bgr8_to_jpeg(image)
    
camera.observe(update_image, names='value')

当在相机前移动某物体时，小组件中应会显示实时视频流。如需停止，可使用 `unobserve` 方法取消附加回调。

In [ ]:
camera.unobserve(update_image, names='value')

<div style="border:2px solid black; background-color:#e3ffb3; font-size:12px; padding:8px; margin-top: auto;"><i>
<h4><i>提示</i></h4>
通过右键单击单元并选择“Create New View for Output”（新建输出视图），您可将小组件（或任何单元）移至 JupyterLab 中的新窗口选项卡。如此一来，您便可继续向下滚动 JupyterLab notebook，同时仍可查看相机视图！
</i></div>

### 查看图像流的另一种方法
此外，您还可使用 traitlets `dlink` 方法将相机连接到小组件，并将转换用作其中一个参数。此方法可免去流程中的一些步骤。

In [ ]:
import traitlets

camera_link = traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)

您可使用 `unlink` 方法删除相机或小组件链接。

In [ ]:
camera_link.unlink()

...然后使用 `link` 方法重新建立连接。

In [ ]:
camera_link.link()

## 关闭此 notebook 的内核以释放相机资源。
返回 DLI 课程页面查看后续说明。

<div style="border:2px solid black; background-color:#e3ffb3; font-size:12px; padding:8px; margin-top: auto;"><i>
<h4><i>提示</i></h4>
<p>一次仅限使用一个 CSICamera 或 USBCamera 实例。如要新建相机实例，必须首先释放现有实例。为此，请在 JupyterLab 下拉菜单中依次单击<strong>“Kernel”（内核）->“Shutdown Kernel”（关闭内核）</strong>以关闭 notebook 内核，然后单击<strong>“Kernel（内核）”->“Restart Kernel”（重启内核）</strong>将其重启。</p>
    <p>如果相机设置出现“停滞”或图像出现“卡顿”，请按以下步骤操作：
    <ol><li>按上述操作关闭 notebook 内核</li>
        <li>单击“Launch”（启动）页面上的“Terminal”（终端）图标，在 Jetson Nano 中打开一个终端窗口</li>
        <li>在终端窗口中输入以下命令：<code>sudo systemctl restart nvargus-daemon</code>，然后输入密码：<code>dlinano</code></li>
    </ol>
    </i></div>

<center><img src="../images/DLI Header.png" alt="标题" style="width: 400px;"/></center>